The goal of this notebook is to take a sample of our tokenized dataset, run it through tf-sentence-transforms, and see if our custom embedding does any better

In [15]:
import pandas as pd
from tf_sentence_transformers import SentenceTransformer
import random
import tensorflow as tf

In [5]:
df = pd.read_pickle('data/tokenized_dataset.pkl')

In [6]:
df.head()

,tune_id,setting_id,name,type,meter,mode,abc,date,username,full_abc,transposed_abc,tokenized_abc,note_seq,dur_seq
0,18105,35234,"$150 Boot, The",polka,2/4,Gmajor,|:d>g fe|dB AG|E/F/G E/F/G|BA GF|\nd>g fe|dB A...,2019-07-06 04:39:09,NfldWhistler,"X:35234\nT:$150 Boot, The\nM:2/4\nK:Gmajor\n|:...","X:35234\nT:$150 Boot, The\nM:2/4\nK:Cmajor\n|:...","[G_0.500, c_0.500, B_0.500, A_0.500, G_0.500, ...","[G, c, B, A, G, E, D, C, A, B, C, A, B, C, E, ...","[0.75, 0.25, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5..."
1,11931,11931,'G Iomain Nan Gamhna,slip jig,9/8,Gmajor,dBB B2 A BAG|dBB Bcd efg|dBB B2 A BAG|eAA dBG ...,2012-05-17 07:49:26,iliketurtles,X:11931\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Gmaj...,X:11931\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Cmaj...,"[G_0.500, E_0.500, E_0.500, E_1.000, D_0.500, ...","[G, E, E, E, D, E, D, C, G, E, E, E, F, G, A, ...","[0.5, 0.5, 0.5, 1.0, 0.5, 0.5, 0.5, 0.5, 0.5, ..."
2,11931,48772,'G Iomain Nan Gamhna,slip jig,9/8,Amixolydian,|:dBB BBA BAG|dBB Bcd efg|dBB BBA BAG|e2A dBG ...,2023-11-25 22:54:00,birlibirdie,X:48772\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Amix...,X:48772\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Cmix...,"[F_0.500, D_0.500, D_0.500, D_0.500, D_0.500, ...","[F, D, D, D, D, C, D, C, B, F, D, D, D, E, F, ...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, ..."
3,15326,28560,'S Ann An Ìle,strathspey,4/4,Gmajor,|:G>A B>G c>A B>G|E<E A>G F<D D2|G>A B>G c>A B...,2016-03-31 15:34:45,danninagh,X:28560\nT:'S Ann An Ìle\nM:4/4\nK:Gmajor\n|:G...,X:28560\nT:'S Ann An Ìle\nM:4/4\nK:Cmajor\n|:C...,"[C_0.500, D_0.500, E_0.500, C_0.500, F_0.500, ...","[C, D, E, C, F, D, E, C, A, A, D, C, B, G, G, ...","[0.75, 0.25, 0.75, 0.25, 0.75, 0.25, 0.75, 0.2..."
4,15326,28582,'S Ann An Ìle,strathspey,4/4,Gmajor,"uD2|:{F}v[G,2G2]uB>ud c>A B>G|{D}E2 uA>uG F<D ...",2016-04-03 09:15:08,DonaldK,X:28582\nT:'S Ann An Ìle\nM:4/4\nK:Gmajor\nuD2...,"X:28582\nT:'S Ann An Ìle\nM:4/4\nK:Cmajor\nuG,...","[G_0.500, B_0.500, C_0.500, C_1.000, E_0.500, ...","[G, B, C, C, E, G, F, D, E, C, G, A, D, C, B, ...","[0.5, 0.5, 0.5, 1.0, 0.75, 0.25, 0.75, 0.25, 0..."


In [7]:
layer = SentenceTransformer.from_pretrained(
    "sentence-transformers/all-MiniLM-L6-v2"
)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
I0000 00:00:1747955203.289867  646592 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3455 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070, pci bus id: 0000:01:00.0, compute capability: 8.9
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a B

In [19]:
def sample_triplet(df: pd.DataFrame) -> list[str]:
    # 1) find all tune_ids with ≥2 distinct settings
    multi_setting = (
        df.groupby('tune_id')['setting_id']
          .nunique()
          .loc[lambda x: x >= 2]
          .index
          .tolist()
    )
    if not multi_setting:
        raise ValueError("No tune_id has ≥2 settings")

    # 2) pick one tune_id for the “positive” pair
    tune_pos = random.choice(multi_setting)
    settings_pos = df.loc[df['tune_id'] == tune_pos, 'setting_id'].unique().tolist()
    s1, s2 = random.sample(settings_pos, 2)

    # grab their ABCs
    abc1 = df.loc[
        (df['tune_id'] == tune_pos) & (df['setting_id'] == s1),
        'transposed_abc'
    ].iloc[0]
    abc2 = df.loc[
        (df['tune_id'] == tune_pos) & (df['setting_id'] == s2),
        'transposed_abc'
    ].iloc[0]

    # 3) pick a different tune for the “negative” example
    other_tunes = df['tune_id'].unique().tolist()
    other_tunes.remove(tune_pos)
    if not other_tunes:
        raise ValueError("No other tune_id available for negative example")

    tune_neg = random.choice(other_tunes)
    setting_neg = random.choice(df.loc[df['tune_id'] == tune_neg, 'setting_id'].unique().tolist())
    abc3 = df.loc[
        (df['tune_id'] == tune_neg) & (df['setting_id'] == setting_neg),
        'transposed_abc'
    ].iloc[0]

    inputs = [[abc1], [abc2], [abc3]]
    return tf.constant(inputs, dtype=tf.string)
# Example usage:
# sampled_abcs = sample_triplet(df)
# sentence_embeddings = model.encode(sampled_abcs)

In [20]:
test = sample_triplet(df)

In [23]:
embedding = layer(test)